#### Считывание csv
- encoding - кодировка ('utf-8', 'ISO-8859-1', 'Windows-1251' и др)
- sep - разделитель (по умолчанию ',')
- compression - метод сжатия для архивов
- parse_dates - стоит ли воспринимать даты как даты (по-умолчанию pandas воспринимает как строки).\n
    parse_dates = True - в дату переводится первая колонка или можно указать массив колонок



In [1]:
import pandas as pd
data  = pd.read_csv('data/6_data.zip', compression='zip', encoding='ISO-8859-1')

####  Первоначальные характеристики датафрейма
- df.**shape** - размерность датафрема (row, column)
- df.**head(n)** - первые n строчек из датафрейма (default=5)
- df.**tail(n)** - последние n строчек из датафрейма (default=5)
- df.**dtypes** - типы данных в столбцах датафрейма
- df.**[имя колонки].dtype** - тип значений в конкретной колонке
- df.**describe()** - рассчитывает базовые метрики для таблицы
- df.**info** - информация о датафрейме (
- df.**columns** - список колонок
- df.**index** - список индексов
- df[['column1', ''column2', 'column3']] - взять определенные столбцы из df

In [2]:
data.shape # число строк и столбцов в датафрейме
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


#### Проверить типы данных в столбцах датафрейма и определить, какой тип данных преобладает
1) Атрибут **dtypes**. Он возвращает серию, индексами которой являются названия столбцов, а значениями — тип данных в этих столбцах. Потом пробежаться глазами и посмотреть самый частый тип
2) Применить к датафрейму метод **info()**, который выводит подробную информацию о датафрейме и его колонках. Смотрим на последнюю строчку, там будет инфа о всех типах, например,
dtypes: float64(2), int64(1), object(5)

In [3]:
data.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


#### Переименование колонок
Для переименования колонок передайте в метод `rename()` в параметр `columns` словарь, где ключами являются старые названия колонок, а значениями — новые названия.

In [ ]:
data = data.rename(columns={'old_name': 'new_name'})
#data.columns

#### Разные методы работы со столбцами
- df.column.**value_counts()** - число записей по каждому значению для колонки. По умолчанию в порядке убывания (**ascending=False**)
- df.column.**sum()** - сумма всех значений в столбце
- df.**sort_values('column', ascending=False)** - сортировка строк по значению в колонке column

In [23]:
data.Country.value_counts().head()

Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Name: count, dtype: int64

#### Группировка и агрегация данных
Для агрегации используется метод **agg**, обычно его используют сразу после группировки через **groupby**.
В результате агрерации из массива значений (колонка) получается одно значение на каждую агрегирующую функцию.
1) Если использовать перадачу в **agg** значений через словарь, то результат будет **датафреймом**.
Словарь вида
```[колонка для группировки]: функция, которую применяем к значениям этой колонки```

**groupby**('column', **as_index = False**) - позволяет не делать колонку индексом в датафрейме после группировки.

In [19]:
data.groupby('Country').agg({'InvoiceNo': 'count'}).head(3)  #количество заказов, сгруппированных по стране

,InvoiceNo
Country,
Australia,1259
Austria,401
Bahrain,19


2. Можно вызывать функцию-агрерат непосредственно на **колонке после группировки**. Тогда результат - **Series** (а-ля колонка в pandas)


In [18]:
data.groupby('Country').InvoiceNo.count().head(3)

Country
Australia    1259
Austria       401
Bahrain        19
Name: InvoiceNo, dtype: int64

####  Поиск минимального и максимального значения
- df.column.**idxmax()** - индекс максимального значения
- df.column.**idxmin()** - индекс минимального значения
- df.column.**min()** - минимальное значение
- df.column.**max()** - максимальное значение

In [20]:
data.Quantity.max()

80995

In [21]:
data.Quantity.idxmax()

540421

In [22]:
data.index

RangeIndex(start=0, stop=541909, step=1)

#### Фильтрация
Есть 2 способа:
1) df.**query**("условие")
логическое И: and или &
логическое ИЛИ: or или |
названия колонок  - без кавычек, строковые значения - в кавычках. Если нужно сравнить значение с переменной, то  **@** перед переменной
2) Метод df.**loc** - в широком смысле дает доступ к группе строк и столбцов по меткам или логическому массиву.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html  - получше разобраться


In [24]:
country = "Germany"
data_by_query = data.query("Country == @country & Quantity > 500")
data_by_loc = data.loc[(data.Country == country) & (data.Quantity > 500)]
data_by_loc

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
187574,552978,15036,ASSORTED COLOURS SILK FAN,600,5/12/2011 14:46,0.65,12590.0,Germany
